In [1]:
import os
print(os.getcwd())

d:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction


## Patch-level feature extraction

In [ ]:
from huggingface_hub import login
from transformers import AutoModel
from PIL import Image
import torch
import numpy

login(token="")
# token = ""

titan = AutoModel.from_pretrained("MahmoodLab/TITAN",trust_remote_code=True)
conch, eval_transform = titan.return_conch()

# Step 1: Load your image
image = Image.open("Example_TCGA-CK-6747_MSIH_x1600_y19792_patch00018.PNG").convert("RGB")

# Step 2: Apply eval_transform from the model
input_tensor = eval_transform(image).unsqueeze(0)  # add batch dimension (1, C, H, W)

# Step 3: Inference using conch model
with torch.no_grad():
    output = conch(input_tensor)

# Step 4: Print or interpret output
# print("Model Output:", output)
print(output.shape)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\datainsight\.cache\huggingface\token
Login successful
torch.Size([1, 768])


## TESTING Slide-level feature extraction

In [3]:
import h5py
import torch

from transformers import AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# load model
model = AutoModel.from_pretrained('MahmoodLab/TITAN', trust_remote_code=True)
model = model.to(device)

In [ ]:
# load data
from huggingface_hub import hf_hub_download
demo_h5_path = r"D:\Titan_Project\Features_Tests\3732_ResNet\features\h5_files\TCGA-AG-3732-01Z-00-DX1.5EC57511-2B19-4005-BCA0-333C387C66E6.h5"
file = h5py.File(demo_h5_path, 'r')
features = torch.from_numpy(file['features'][:])
coords = torch.from_numpy(file['coords'][:])
patch_size_lv0 = 1024 # Change this if needed

In [ ]:
# Convert tensors to proper format
coords = coords.to(torch.int64)
coords = coords.unsqueeze(0)
features = features.unsqueeze(0)
print(features.shape)
print(coords.shape)


torch.Size([1, 416, 1024])
torch.Size([1, 416, 2])


### Temporary fix (Project 1024D feature vector to 768D)

In [61]:
projector = torch.nn.Linear(1024, 768).to(device)
features = projector(features)

In [62]:
print(features.shape)
print(coords.shape)
print(features.dtype)
print(coords.dtype)

torch.Size([1, 416, 768])
torch.Size([1, 416, 2])
torch.float32
torch.int64


In [63]:
# extract slide embedding
with torch.inference_mode():
    features = features.to(device)
    coords = coords.to(device)
    slide_embedding = model.encode_slide_from_patch_features(features, coords, patch_size_lv0)

In [64]:
print(f"Success! Slide embedding shape: {slide_embedding.shape}")
slide_embedding

Success! Slide embedding shape: torch.Size([1, 768])


tensor([[-0.3507,  0.1118,  0.2407, -0.2569, -0.1810, -0.3236,  0.2718,  0.1074,
          0.2211,  0.0543, -0.1086, -0.4009,  0.5252, -0.5609,  0.5233,  0.2071,
          0.4965,  0.1718,  0.0791, -0.3842,  0.2353,  0.1580, -0.2563, -0.2422,
          0.0682, -0.1396, -0.2234, -0.1133, -0.1999, -0.0227,  0.0115, -0.0191,
          0.0412, -0.1559,  0.0707, -0.4688, -0.5219,  0.2895, -0.0920,  0.0390,
          0.1521,  0.1056, -0.0167, -0.4385, -0.0811, -0.3525,  0.2790, -0.0095,
          0.0869,  0.0987, -0.6399,  0.1056, -0.3872,  0.4995, -0.4174,  0.1354,
          0.2928, -0.2555, -0.0299,  0.5408,  0.0264,  0.2145, -0.1215, -0.2158,
          0.3432,  0.4145,  0.2335, -0.0012,  0.2497, -0.6105,  0.0485,  0.0158,
         -0.1695, -0.7591,  0.6008, -0.2165,  0.0170, -0.4040, -0.1779, -0.5484,
          0.7142, -0.0909,  0.3025, -0.0034, -0.2406,  0.2683, -0.2527,  0.2805,
          0.4486,  0.2429, -0.3397,  0.7294,  0.2538,  0.3722,  0.5084,  0.4525,
          0.2945, -0.5134,  

In [2]:
import h5py

def inspect_h5_file(h5_path):
    """Inspect the contents of an H5 file"""
    print(f"Inspecting H5 file: {h5_path}")
    print("=" * 50)
    
    with h5py.File(h5_path, 'r') as file:
        def print_structure(name, obj):
            if isinstance(obj, h5py.Dataset):
                print(f"Dataset: {name}")
                print(f"  Shape: {obj.shape}")
                print(f"  Dtype: {obj.dtype}")
                if hasattr(obj, 'attrs') and len(obj.attrs) > 0:
                    print(f"  Attributes: {dict(obj.attrs)}")
                print()
            elif isinstance(obj, h5py.Group):
                print(f"Group: {name}")
                if hasattr(obj, 'attrs') and len(obj.attrs) > 0:
                    print(f"  Attributes: {dict(obj.attrs)}")
                print()
        
        file.visititems(print_structure)

# First, inspect your H5 file
# demo_h5_path = "TCGA-PC-A5DK-01Z-00-DX1.C2D3BC09-411F-46CF-811B-FDBA7C2A295B.h5"
# h5_path = r"D:\Titan_Project\Features_Tests\3732_ResNet\features\h5_files\TCGA-AG-3732-01Z-00-DX1.5EC57511-2B19-4005-BCA0-333C387C66E6.h5"
h5_path = r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5\TCGA-3L-AA1B_nonMSIH\patch_features\TCGA-3L-AA1B_nonMSIH_patch_features.h5"
inspect_h5_file(h5_path)
# inspect_h5_file(demo_h5_path)

Inspecting H5 file: D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5\TCGA-3L-AA1B_nonMSIH\patch_features\TCGA-3L-AA1B_nonMSIH_patch_features.h5
Dataset: coords
  Shape: (1, 1878, 2)
  Dtype: int64
  Attributes: {'contour_fn': 'four_pt', 'custom_downsample': 2.0, 'downsample': array([1., 1.]), 'name': 'TCGA-3L-AA1B_nonMSIH', 'patch_level': 0, 'patch_size': 1024, 'patch_size_level0': 1024, 'step_size': 1024, 'use_padding': True}

Dataset: features
  Shape: (1, 1878, 768)
  Dtype: float32



In [4]:
import h5py
import numpy as np
import os

def export_features_and_coords(h5_path, output_dir=None):
    # Default output directory is same as h5 file
    if output_dir is None:
        output_dir = os.path.dirname(h5_path)
    os.makedirs(output_dir, exist_ok=True)

    with h5py.File(h5_path, 'r') as f:
        # Read and reshape datasets
        features = f["features"][0]  # shape: (1878, 768)
        coords = f["coords"][0]      # shape: (1878, 2)

    # Define output file paths
    base_name = os.path.splitext(os.path.basename(h5_path))[0]
    features_txt_path = os.path.join(output_dir, f"{base_name}_features.txt")
    coords_txt_path = os.path.join(output_dir, f"{base_name}_coords.txt")

    # Save as plain text
    np.savetxt(features_txt_path, features, fmt="%.6f")
    np.savetxt(coords_txt_path, coords, fmt="%d")

    print(f"✅ Saved features to: {features_txt_path}")
    print(f"✅ Saved coords to: {coords_txt_path}")


h5_path = r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5\TCGA-3L-AA1B_nonMSIH\patch_features\TCGA-3L-AA1B_nonMSIH_patch_features.h5"
export_features_and_coords(h5_path)

# inspect_h5_file(demo_h5_path)

✅ Saved features to: D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5\TCGA-3L-AA1B_nonMSIH\patch_features\TCGA-3L-AA1B_nonMSIH_patch_features_features.txt
✅ Saved coords to: D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5\TCGA-3L-AA1B_nonMSIH\patch_features\TCGA-3L-AA1B_nonMSIH_patch_features_coords.txt


In [5]:
import torch

def inspect_pt_file(pt_path):
    """Inspect the contents of a PyTorch .pt file"""
    print(f"Inspecting PT file: {pt_path}")
    print("=" * 50)
    
    data = torch.load(pt_path, map_location='cpu')
    
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"Key: {key}")
            if isinstance(value, torch.Tensor):
                print(f"  Type: Tensor")
                print(f"  Shape: {tuple(value.shape)}")
                print(f"  Dtype: {value.dtype}")
            else:
                print(f"  Type: {type(value)}")
                print(f"  Value: {value}")
            print()
    else:
        print("The .pt file is not a dictionary. Type:", type(data))
        print("Value preview:", str(data)[:1500])  # Print partial value


pt_path = r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5_patchbypatch\TCGA-3L-AA1B_nonMSIH\TCGA-3L-AA1B_nonMSIH_x2608_y14048.pt"
inspect_pt_file(pt_path)

Inspecting PT file: D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5_patchbypatch\TCGA-3L-AA1B_nonMSIH\TCGA-3L-AA1B_nonMSIH_x2608_y14048.pt
The .pt file is not a dictionary. Type: <class 'torch.Tensor'>
Value preview: tensor([ 1.2918e-01,  1.8351e+00, -2.6008e+00, -1.3341e+00, -5.8160e-01,
         5.1121e-01,  1.1228e+00, -6.3811e-01, -2.3847e+00, -9.4206e-01,
         2.8296e-01,  7.9296e-01, -1.1196e+00,  1.0232e+00,  1.3321e+00,
         4.6913e-02,  3.5359e-01, -1.1387e-01, -4.8670e-01, -1.4748e+00,
        -4.9354e-01, -7.4797e-01,  1.6179e+00,  2.0795e-01, -2.0888e+00,
         3.9398e-01,  4.9112e-01,  3.8618e-01,  5.9097e-01, -8.7534e-01,
        -1.9093e+00,  2.9548e-01,  6.1419e-01,  6.7622e-01, -6.7314e-01,
         8.5244e-01, -1.6878e+00,  1.3609e+00,  7.5817e-01, -7.3301e-01,
        -4.7759e-02, -7.2221e-02, -9.5563e-01, -1.6662e+00, -4.2791e-01,
        -5.3904e-01, -5.7638e-01,  7.7261e-01,  7.4532e-01,  4.7517e-01,
         4.4111e-01,  2.1106e-01, -7.3000e-0

## Feature extraction using CLAM with Conch_v1_5(H5 file creation)

In [5]:
# Test if conch_v1_5 working
from transformers import AutoModel
titan = AutoModel.from_pretrained("MahmoodLab/TITAN", trust_remote_code=True)
model, _ = titan.return_conch()

print(type(model))

<class 'transformers_modules.MahmoodLab.TITAN.b2fb4f475256eb67c6e9ccbf2d6c9c3f25f20791.conch_v1_5.EncoderWithAttentionalPooler'>


In [ ]:
!python extract_features_fp.py \
  --data_h5_dir DIR_TO_COORDS \
  --data_slide_dir DATA_DIRECTORY \
  --csv_path process_list_autogen.csv \
  --feat_dir features \
  --batch_size 512 \
  --slide_ext .svs \
  --model_name conch_v1_5 \
  --target_patch_size 448

# check if script setup properly
# !python extract_features_fp.py \
# --help


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [02:25<?, ?it/s]

  0%|          | 0/1 [02:25<?, ?it/s]
Traceback (most recent call last):
  File "d:\Titan_Project\extract_features_fp.py", line 105, in <module>
    output_file_path = compute_w_loader(output_path, loader = loader, model = model, verbose = 1)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Titan_Project\extract_features_fp.py", line 41, in compute_w_loader
    features = model(batch)
               ^^^^^^^^^^^^
  File "d:\miniconda_3\envs\titan_env\Lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Hp/.cache\huggingface\modules\transformers_modules\MahmoodLab\TITAN\b2fb4f475256eb67c6e9ccbf2d6c9c3f25f20791\conch_v1_5.py", line 657, in forward
    x = self.trunk(x, return_all_tokens=True)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

initializing dataset
loading model checkpoint
EncoderWithAttentionalPooler(
  (trunk): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1024, out_features=3072, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1024, out_features=1024, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=1024, 

In [ ]:
#Set model checkpoint for conch_v_1 (not needed if using conchv1_5)
os.environ['CONCH_CKPT_PATH'] = 'checkpoints/conch/pytorch_model.bin'

## Feature extraction using CLAM with Uni_v1(H5 file creation) 

In [13]:
import os

# Set it directly in code
os.environ["UNI_CKPT_PATH"] = "D:\\Titan_Project\\checkpoints\\UNI\\pytorch_model.bin"

# Now access it
ckpt_path = os.environ["UNI_CKPT_PATH"]
print(ckpt_path)

D:\Titan_Project\checkpoints\UNI\pytorch_model.bin


In [15]:
!python extract_features_fp.py \
  --data_h5_dir DIR_TO_COORDS \
  --data_slide_dir DATA_DIRECTORY \
  --csv_path process_list_autogen.csv \
  --feat_dir features \
  --batch_size 512 \
  --slide_ext .svs \
  --model_name uni_v1

initializing dataset
loading model checkpoint
VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate='none')
        (drop1): 


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [19:41<00:00, 1181.02s/it]

100%|██████████| 1/1 [19:41<00:00, 1181.54s/it]


## Patch level Feature extraction using UNI_V2

In [ ]:
import torch
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import timm
from huggingface_hub import login

# Login to HuggingFace (only required once per session)
login(token="")  # Paste your token when prompted

# Load the model with custom parameters
timm_kwargs = {
    'img_size': 224,
    'patch_size': 14,
    'depth': 24,
    'num_heads': 24,
    'init_values': 1e-5,
    'embed_dim': 1536,
    'mlp_ratio': 2.66667 * 2,
    'num_classes': 0,  # So it returns features instead of classification
    'no_embed_class': True,
    'mlp_layer': timm.layers.SwiGLUPacked,
    'act_layer': torch.nn.SiLU,
    'reg_tokens': 8,
    'dynamic_img_size': True
}

# Load model
model = timm.create_model("hf-hub:MahmoodLab/UNI2-h", pretrained=True, **timm_kwargs)
model.eval()

# Set transform based on the model's config
transform = create_transform(**resolve_data_config(model.pretrained_cfg, model=model))

# Load and transform an image
image = Image.open("Example_patch_00029_x2699_y27842.png").convert("RGB")
input_tensor = transform(image).unsqueeze(0)  # (1, 3, 224, 224)

# Inference (feature extraction)
with torch.no_grad():
    features = model(input_tensor)  # Output shape: (1, 1536)
print("Feature shape:", features.shape)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\Hp\.cache\huggingface\token
Login successful
Feature shape: torch.Size([1, 1536])


## Feature extraction using CLAM with Resnet50

In [ ]:
!python extract_features_fp.py \
  --data_h5_dir DIR_TO_COORDS \
  --data_slide_dir DATA_DIRECTORY \
  --csv_path process_list_autogen.csv \
  --feat_dir features \
  --batch_size 512 \
  --slide_ext .svs

In [4]:
# check if script setup properly
!python extract_features_fp.py \
--help

usage: extract_features_fp.py [-h] [--data_h5_dir DATA_H5_DIR]
                              [--data_slide_dir DATA_SLIDE_DIR]
                              [--slide_ext SLIDE_EXT] [--csv_path CSV_PATH]
                              [--feat_dir FEAT_DIR]
                              [--model_name {resnet50_trunc,uni_v1,conch_v1,conch_v1_5}]
                              [--batch_size BATCH_SIZE] [--no_auto_skip]
                              [--target_patch_size TARGET_PATCH_SIZE]

Feature Extraction

options:
  -h, --help            show this help message and exit
  --data_h5_dir DATA_H5_DIR
  --data_slide_dir DATA_SLIDE_DIR
  --slide_ext SLIDE_EXT
  --csv_path CSV_PATH
  --feat_dir FEAT_DIR
  --model_name {resnet50_trunc,uni_v1,conch_v1,conch_v1_5}
  --batch_size BATCH_SIZE
  --no_auto_skip
  --target_patch_size TARGET_PATCH_SIZE


## Pipeline for Conchv_1_5 with TITAN for Slide-level Embedding

In [4]:
# Test if conch_v1_5 working
from transformers import AutoModel
titan = AutoModel.from_pretrained("MahmoodLab/TITAN", trust_remote_code=True)
model, _ = titan.return_conch()

print(type(model))

<class 'transformers_modules.MahmoodLab.TITAN.b2fb4f475256eb67c6e9ccbf2d6c9c3f25f20791.conch_v1_5.EncoderWithAttentionalPooler'>


In [15]:
# Extract patch features using CONCH v1.5 (Patch-level feature extraction)
!python extract_features_fp.py \
  --data_h5_dir DIR_TO_COORDS \
  --data_slide_dir DATA_DIRECTORY \
  --csv_path process_list_autogen.csv \
  --feat_dir features \
  --batch_size 256 \
  --slide_ext .svs \
  --model_name conch_v1_5 \
  --target_patch_size 448


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [01:04<?, ?it/s]

  0%|          | 0/1 [01:04<?, ?it/s]
Traceback (most recent call last):
  File "d:\Titan_Project\extract_features_fp.py", line 111, in <module>
    output_file_path = compute_w_loader(output_path, loader = loader, model = model, verbose = 1)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Titan_Project\extract_features_fp.py", line 43, in compute_w_loader
    features = model(batch)
               ^^^^^^^^^^^^
  File "d:\miniconda_3\envs\titan_env\Lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Hp/.cache\huggingface\modules\transformers_modules\MahmoodLab\TITAN\b2fb4f475256eb67c6e9ccbf2d6c9c3f25f20791\conch_v1_5.py", line 657, in forward
    x = self.trunk(x, return_all_tokens=True)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

initializing dataset
loading model checkpoint
EncoderWithAttentionalPooler(
  (trunk): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1024, out_features=3072, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1024, out_features=1024, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=1024, 

In [ ]:
import os
import torch
import h5py
from tqdm import tqdm
from transformers import AutoModel

# --- Config ---
h5_dir = r"D:\Titan_Project\features\h5_files"
save_dir = r"D:\Titan_Project\features\slide_embeddings"
os.makedirs(save_dir, exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- Load TITAN Model ---
print("Loading TITAN model...")
model = AutoModel.from_pretrained('MahmoodLab/TITAN', trust_remote_code=True).to(device)
model.eval()

# --- Iterate over all .h5 files ---
h5_files = [f for f in os.listdir(h5_dir) if f.endswith('.h5')]

for h5_file in tqdm(h5_files, desc="Processing slides"):
    h5_path = os.path.join(h5_dir, h5_file)

    try:
        with h5py.File(h5_path, 'r') as f:
            features = torch.from_numpy(f['features'][:])  # (N, 768)
            coords = torch.from_numpy(f['coords'][:])      # (N, 2)
            patch_size_lv0 = f['coords'].attrs.get('patch_size_level0', 1024)

        features = features.unsqueeze(0).to(device)  # (1, N, 768)
        coords = coords.unsqueeze(0).to(device)      # (1, N, 2)

        # Extract slide embedding
        with torch.inference_mode(), torch.autocast('cuda', torch.float16):
            slide_embedding = model.encode_slide_from_patch_features(features, coords, patch_size_lv0)

        # Save embedding using first 12 characters (e.g., TCGA-AG-3732)
        case_id = h5_file[:12]
        save_path = os.path.join(save_dir, f"{case_id}.pt")
        torch.save(slide_embedding.cpu(), save_path)

    except Exception as e:
        print(f"❌ Failed on {h5_file}: {e}")


## Alternative method using donwsampled PNG by PNG for feature extraction

### Feature extraction function

In [ ]:
from huggingface_hub import login
from transformers import AutoModel
from PIL import Image
import torch
import numpy as np
from typing import Union, List
import os

class ConchFeatureExtractor:
    def __init__(self, token: str = ""):
        """
        Initialize the Conch feature extractor.
        
        Args:
            token: HuggingFace token for authentication
        """
        login(token=token)
        self.model = AutoModel.from_pretrained("MahmoodLab/TITAN", trust_remote_code=True)
        self.conch, self.eval_transform = self.model.return_conch()
        
        # Move model to GPU if available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.conch.to(self.device)
        self.conch.eval()  # Set to evaluation mode
    
    def extract_features(self, 
                        images: Union[str, Image.Image, List[str], List[Image.Image]], 
                        batch_size: int = 32) -> np.ndarray:
        """
        Extract features from single image or batch of images.
        
        Args:
            images: Single image path/PIL Image or list of image paths/PIL Images
            batch_size: Batch size for processing multiple images
            
        Returns:
            numpy array of shape (n_images, feature_dim) containing feature vectors
        """
        # Handle single image case
        if isinstance(images, (str, Image.Image)):
            images = [images]
        
        # Load and preprocess images
        image_tensors = []
        for img in images:
            if isinstance(img, str):
                # Load from path
                if not os.path.exists(img):
                    raise FileNotFoundError(f"Image not found: {img}")
                pil_img = Image.open(img).convert("RGB")
            elif isinstance(img, Image.Image):
                # Use PIL Image directly
                pil_img = img.convert("RGB")
            else:
                raise ValueError(f"Unsupported image type: {type(img)}")
            
            # Apply transformation
            tensor = self.eval_transform(pil_img)
            image_tensors.append(tensor)
        
        # Stack all tensors
        batch_tensor = torch.stack(image_tensors)
        
        # Process in batches
        all_features = []
        
        with torch.no_grad():
            for i in range(0, len(batch_tensor), batch_size):
                batch = batch_tensor[i:i + batch_size].to(self.device)
                features = self.conch(batch)
                
                # Move to CPU and convert to numpy
                features_np = features.cpu().numpy()
                all_features.append(features_np)
        
        # Concatenate all batches
        result = np.concatenate(all_features, axis=0)
        
        return result

# Convenience function for quick usage
def extract_titan_features(images: Union[str, Image.Image, List[str], List[Image.Image]], 
                          batch_size: int = 32,
                          token: str = "") -> np.ndarray:
    """
    Quick function to extract Conch features without class instantiation.
    
    Args:
        images: Single image path/PIL Image or list of image paths/PIL Images
        batch_size: Batch size for processing multiple images
        token: HuggingFace token for authentication
        
    Returns:
        numpy array of shape (n_images, feature_dim) containing feature vectors
    """
    extractor = ConchFeatureExtractor(token=token)
    return extractor.extract_features(images, batch_size)

### Feature extraction with fivecrop function

In [ ]:
from huggingface_hub import login
from transformers import AutoModel
from PIL import Image
import torch
import numpy as np
from torchvision import transforms
from typing import Union, List
import os

class ConchFeatureExtractorFiveCrop:
    def __init__(self, token: str = ""):
        login(token=token)
        self.model = AutoModel.from_pretrained("MahmoodLab/TITAN", trust_remote_code=True)
        self.conch, self.eval_transform = self.model.return_conch()

        # Add FiveCrop wrapper
        self.fivecrop_transform = transforms.Compose([
            # transforms.Resize(256),
            transforms.FiveCrop(224),  # yields tuple of 5 images
            transforms.Lambda(lambda crops: torch.stack([self.eval_transform(c) for c in crops]))
        ])

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.conch.to(self.device)
        self.conch.eval()

    def extract_fivecrop_features(self, image: Union[str, Image.Image]) -> torch.Tensor:
        """
        Extract 5xD features from a single image using FiveCrop.
        Returns a tensor of shape (5, D).
        """
        if isinstance(image, str):
            if not os.path.exists(image):
                raise FileNotFoundError(f"Image not found: {image}")
            image = Image.open(image).convert("RGB")
        elif isinstance(image, Image.Image):
            image = image.convert("RGB")
        else:
            raise ValueError(f"Unsupported image type: {type(image)}")

        crops_tensor = self.fivecrop_transform(image)  # shape: (5, C, H, W)
        crops_tensor = crops_tensor.to(self.device)

        with torch.no_grad():
            features = self.conch(crops_tensor)  # shape: (5, D)

        return features.cpu()  # return to CPU for saving
    

### Testing for feature extraction

In [ ]:
# FiveCrop feature extraction function
import glob
import os
from tqdm import tqdm
import torch

# Set paths
patch_root = r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\test_data"
save_root = r"D:\Aamir Gulzar\KSA_project2\dataset\Features_test\Conch_v1_5_fivecrop"

# Initialize extractor
extractor = ConchFeatureExtractorFiveCrop()

# Loop through all PNG patches
all_patch_paths = glob.glob(os.path.join(patch_root, "*", "*.png"))

for patch_path in tqdm(all_patch_paths):
    try:
        features = extractor.extract_fivecrop_features(patch_path)  # shape: (5, D)

        # Extract subfolder and filename
        relative_path = os.path.relpath(patch_path, patch_root)
        slide_folder, patch_filename = os.path.split(relative_path)
        patch_base = os.path.splitext(patch_filename)[0]  # remove .png

        # Create save path
        save_dir = os.path.join(save_root, slide_folder)
        os.makedirs(save_dir, exist_ok=True)

        save_path = os.path.join(save_dir, patch_base + ".pt")
        torch.save(features, save_path)
    
    except Exception as e:
        print(f"Error processing {patch_path}: {e}")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\datainsight\.cache\huggingface\token
Login successful


100%|██████████| 6695/6695 [13:38<00:00,  8.18it/s]


In [ ]:
# Initialize extractor (recommended for multiple calls)
extractor = ConchFeatureExtractorFiveCrop()

# Single image
single_features = extractor.extract_features("Example_TCGA-CK-6747_MSIH_x1600_y19792_patch00018.PNG")
print(f"Single image features shape: {single_features.shape}")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\Hp\.cache\huggingface\token
Login successful
Single image features shape: (1, 768)


In [4]:
# Multiple images
image_paths = [
    "patches_png\patch_00044.png",
    "patches_png\patch_00054.png"
]
batch_features = extractor.extract_features(image_paths, batch_size=16)
print(f"Batch features shape: {batch_features.shape}")

Batch features shape: (2, 768)


### pipeline version 1 for feature extraction into h5

In [ ]:
import h5py
import numpy as np
import re
import os
from typing import List, Tuple, Dict, Any
from pathlib import Path
import glob

def extract_coords_from_filename(filename: str) -> Tuple[int, int]:
    """
    Extract x, y coordinates from patch filename.
    
    Expected format: TCGA-AA-3846_nonMSIH_x3152_y22080_patch00015.png
    
    Args:
        filename: Full filename or just the basename
        
    Returns:
        Tuple of (x, y) coordinates
        
    Raises:
        ValueError: If coordinates cannot be extracted from filename
    """
    # Remove path and extension to get just the basename
    basename = os.path.splitext(os.path.basename(filename))[0]
    
    # Pattern to match x and y coordinates
    pattern = r'_x(\d+)_y(\d+)'
    
    match = re.search(pattern, basename)
    if not match:
        raise ValueError(f"Could not extract coordinates from filename: {filename}")
    
    x_coord = int(match.group(1))
    y_coord = int(match.group(2))
    
    return x_coord, y_coord

def create_h5_from_features(
    features: np.ndarray,
    image_paths: List[str],
    output_path: str,
    slide_name: str = None,
    patch_size_level0: int = 1024,
    additional_attributes: Dict[str, Any] = None
) -> str:
    """
    Create H5 file with features and coordinates extracted from image filenames.
    
    Args:
        features: Feature array of shape (n_patches, feature_dim)
        image_paths: List of image file paths corresponding to features
        output_path: Path where to save the H5 file
        slide_name: Name of the slide (extracted from first filename if None)
        patch_size_level0: Patch size at level 0
        additional_attributes: Additional attributes to add to coords dataset
        
    Returns:
        Path to created H5 file
    """
    if len(features) != len(image_paths):
        raise ValueError(f"Features length ({len(features)}) must match image_paths length ({len(image_paths)})")
    
    # Extract coordinates from filenames
    coords = []
    for img_path in image_paths:
        try:
            x, y = extract_coords_from_filename(img_path)
            coords.append([x, y])
        except ValueError as e:
            print(f"Warning: {e}")
            # Skip this image or use default coordinates
            continue
    
    coords = np.array(coords, dtype=np.int64)
    
    # If slide_name not provided, extract from first filename
    if slide_name is None:
        first_filename = os.path.basename(image_paths[0])
        # Extract slide name (everything before first underscore that's not TCGA)
        parts = first_filename.split('_')
        if len(parts) >= 2:
            slide_name = '_'.join(parts[:2])  # e.g., "TCGA-AA-3846_nonMSIH"
        else:
            slide_name = parts[0]
    
    # Ensure features have the right shape (add batch dimension if needed)
    if features.ndim == 2:
        features = features[np.newaxis, :]  # Add batch dimension: (1, n_patches, feature_dim)
    
    # Ensure coords have the right shape
    if coords.ndim == 2:
        coords = coords[np.newaxis, :]  # Add batch dimension: (1, n_patches, 2)
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Create H5 file
    with h5py.File(output_path, 'w') as h5f:
        # Create features dataset
        features_dataset = h5f.create_dataset(
            'features', 
            data=features.astype(np.float32),
            dtype=np.float32
        )
        
        # Create coordinates dataset
        coords_dataset = h5f.create_dataset(
            'coords',
            data=coords.astype(np.int64),
            dtype=np.int64
        )
        
        # Add attributes to coords dataset
        default_attributes = {
            'contour_fn': 'four_pt',
            'custom_downsample': 2.0,
            'downsample': np.array([1., 1.]),
            'name': slide_name,
            'patch_level': 0,
            'patch_size': patch_size_level0,
            'patch_size_level0': patch_size_level0,
            'step_size': patch_size_level0,
            'use_padding': True
        }
        
        # Add additional attributes if provided
        if additional_attributes:
            default_attributes.update(additional_attributes)
        
        # Set defualt attributes if not provided
        # for key, value in default_attributes.items():
        #     coords_dataset.attrs[key] = value
    
    print(f"H5 file created successfully: {output_path}")
    print(f"Features shape: {features.shape}")
    print(f"Coords shape: {coords.shape}")
    
    return output_path

def process_patch_directory(
    patch_dir: str,
    extractor,
    output_h5_path: str,
    batch_size: int = 32,
    file_pattern: str = "*.png",
    patch_size_level0: int = 1024
) -> str:
    """
    Process all patches in a directory and create H5 file.
    
    Args:
        patch_dir: Directory containing patch images
        extractor: ConchFeatureExtractor instance
        output_h5_path: Path for output H5 file
        batch_size: Batch size for feature extraction
        file_pattern: File pattern to match (e.g., "*.png", "*.jpg")
        patch_size_level0: Patch size at level 0
        
    Returns:
        Path to created H5 file
    """
    # Get all image files
    image_paths = glob.glob(os.path.join(patch_dir, file_pattern))
    
    if not image_paths:
        raise ValueError(f"No images found in {patch_dir} with pattern {file_pattern}")
    
    # Sort paths for consistent ordering
    image_paths.sort()
    
    print(f"Found {len(image_paths)} images to process")
    
    # Extract features
    print("Extracting features...")
    features = extractor.extract_features(image_paths, batch_size=batch_size)
    
    # Create H5 file
    print("Creating H5 file...")
    return create_h5_from_features(
        features=features,
        image_paths=image_paths,
        output_path=output_h5_path,
        patch_size_level0=patch_size_level0
    )

# Example usage and testing
if __name__ == "__main__":
    # Test coordinate extraction
    # test_filenames = [
    #     "TCGA-AA-3846_nonMSIH_x3152_y22080_patch00015.png",
    #     "TCGA-PC-A5DK-01Z-00-DX1_x1024_y2048_patch00001.png",
    #     "Example_patch_00029_x2699_y27842.PNG"
    # ]
    
    # print("Testing coordinate extraction:")
    # for filename in test_filenames:
    #     try:
    #         x, y = extract_coords_from_filename(filename)
    #         print(f"{filename} -> x={x}, y={y}")
    #     except ValueError as e:
    #         print(f"Error with {filename}: {e}")
    
    # Example of creating H5 file
    
    # Assuming you have your extractor and image paths

    extractor = ConchFeatureExtractor()
    
    # Process single directory
    h5_path = process_patch_directory(
        patch_dir="patches_png",
        extractor=extractor,
        output_h5_path="output_dir/slide_features.h5",
        batch_size=16
    )
    
    # # Or create from existing features and paths
    # features = np.random.rand(100, 768)  # Example features
    # image_paths = ["patch_x{}_y{}_001.png".format(i*1024, j*1024) 
    #                for i in range(10) for j in range(10)]
    # create_h5_from_features(
    #     features=features,
    #     image_paths=image_paths,
    #     output_path="example_features.h5",
    #     slide_name="TCGA-AA-3846-01A-01-TS1"
    # )
    

Using device: cuda
11.8
True
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\datainsight\.cache\huggingface\token
Login successful
Found 45 images to process
Extracting features...
Creating H5 file...
H5 file created successfully: output_dir/slide_features.h5
Features shape: (1, 45, 768)
Coords shape: (1, 45, 2)


### Testing pipeline version 1

In [ ]:
# Testing 
if __name__ == "__main__":
    # Test coordinate extraction
    # test_filenames = [
    #     "TCGA-AA-3846_nonMSIH_x3152_y22080_patch00015.png",
    #     "TCGA-PC-A5DK-01Z-00-DX1_x1024_y2048_patch00001.png",
    #     "Example_patch_00029_x2699_y27842.PNG"
    # ]
    
    # print("Testing coordinate extraction:")
    # for filename in test_filenames:
    #     try:
    #         x, y = extract_coords_from_filename(filename)
    #         print(f"{filename} -> x={x}, y={y}")
    #     except ValueError as e:
    #         print(f"Error with {filename}: {e}")
    
    # Example of creating H5 file
    
    # Assuming you have your extractor and image paths
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    print(torch.version.cuda)           # Should print CUDA version (e.g., '11.8')
    print(torch.cuda.is_available())    # Should be True


    extractor = ConchFeatureExtractor()
    
    # Process single directory
    h5_path = process_patch_directory(
        patch_dir="patches_png",
        extractor=extractor,
        output_h5_path="output_dir/slide_features.h5",
        batch_size=16
    )
    
    # # Or create from existing features and paths
    # features = np.random.rand(100, 768)  # Example features
    # image_paths = ["patch_x{}_y{}_001.png".format(i*1024, j*1024) 
    #                for i in range(10) for j in range(10)]
    # create_h5_from_features(
    #     features=features,
    #     image_paths=image_paths,
    #     output_path="example_features.h5",
    #     slide_name="TCGA-AA-3846-01A-01-TS1"
    # )

In [10]:
inspect_h5_file("output_dir/slide_features.h5")

Inspecting H5 file: output_dir/slide_features.h5
Dataset: coords
  Shape: (1, 45, 2)
  Dtype: int64

Dataset: features
  Shape: (1, 45, 768)
  Dtype: float32



### pipeline version 2 for feature extraction into h5

In [ ]:
import h5py
import numpy as np
import re
import os
from typing import List, Tuple, Dict, Any
from pathlib import Path
import glob

def extract_coords_from_filename(filename: str) -> Tuple[int, int]:
    """
    Extract x, y coordinates from patch filename.
    
    Expected format: TCGA-AA-3846_nonMSIH_x3152_y22080_patch00015.png
    
    Args:
        filename: Full filename or just the basename
        
    Returns:
        Tuple of (x, y) coordinates
        
    Raises:
        ValueError: If coordinates cannot be extracted from filename
    """
    # Remove path and extension to get just the basename
    basename = os.path.splitext(os.path.basename(filename))[0]
    
    # Pattern to match x and y coordinates
    pattern = r'_x(\d+)_y(\d+)'
    
    match = re.search(pattern, basename)
    if not match:
        raise ValueError(f"Could not extract coordinates from filename: {filename}")
    
    x_coord = int(match.group(1))
    y_coord = int(match.group(2))
    
    return x_coord, y_coord

def create_h5_from_features(
    features: np.ndarray,
    image_paths: List[str],
    output_dir: str,
    slide_name: str = None,
    patch_size_level0: int = 1024,
    additional_attributes: Dict[str, Any] = None
) -> str:
    """
    Create H5 file with features and coordinates extracted from image filenames.
    
    Args:
        features: Feature array of shape (n_patches, feature_dim)
        image_paths: List of image file paths corresponding to features
        output_dir: Directory where to save the H5 file
        slide_name: Name of the slide (extracted from first filename if None)
        patch_size_level0: Patch size at level 0
        additional_attributes: Additional attributes to add to coords dataset
        
    Returns:
        Path to created H5 file
    """
    if len(features) != len(image_paths):
        raise ValueError(f"Features length ({len(features)}) must match image_paths length ({len(image_paths)})")
    
    # Extract coordinates from filenames
    coords = []
    for img_path in image_paths:
        try:
            x, y = extract_coords_from_filename(img_path)
            coords.append([x, y])
        except ValueError as e:
            print(f"Warning: {e}")
            # Skip this image or use default coordinates
            continue
    
    coords = np.array(coords, dtype=np.int64)
    
    # Extract slide name and label from first filename for H5 filename
    if slide_name is None:
        first_filename = os.path.basename(image_paths[0])
        # Extract first 12 characters + label: TCGA-AA-3846_nonMSIH or TCGA-AA-3846_MSIH
        parts = first_filename.split('_')
        if len(parts) >= 2:
            slide_name = '_'.join(parts[:2])  # e.g., "TCGA-AA-3846_nonMSIH"
        else:
            slide_name = parts[0]
    
    # Create H5 filename from slide name
    h5_filename = f"{slide_name}.h5"
    output_path = os.path.join(output_dir, h5_filename)
    
    # Ensure features have the right shape (add batch dimension if needed)
    if features.ndim == 2:
        features = features[np.newaxis, :]  # Add batch dimension: (1, n_patches, feature_dim)
    
    # Ensure coords have the right shape
    if coords.ndim == 2:
        coords = coords[np.newaxis, :]  # Add batch dimension: (1, n_patches, 2)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Create H5 file
    with h5py.File(output_path, 'w') as h5f:
        # Create features dataset
        features_dataset = h5f.create_dataset(
            'features', 
            data=features.astype(np.float32),
            dtype=np.float32
        )
        
        # Create coordinates dataset
        coords_dataset = h5f.create_dataset(
            'coords',
            data=coords.astype(np.int64),
            dtype=np.int64
        )
        
        # Add attributes to coords dataset
        default_attributes = {
            'contour_fn': 'four_pt',
            'custom_downsample': 2.0,
            'downsample': np.array([1., 1.]),
            'name': slide_name,
            'patch_level': 0,
            'patch_size': patch_size_level0,
            'patch_size_level0': patch_size_level0,
            'step_size': patch_size_level0,
            'use_padding': True
        }
        
        # Add additional attributes if provided
        if additional_attributes:
            default_attributes.update(additional_attributes)
        
        # Set attributes
        for key, value in default_attributes.items():
            coords_dataset.attrs[key] = value
    
    print(f"H5 file created successfully: {output_path}")
    print(f"Features shape: {features.shape}")
    print(f"Coords shape: {coords.shape}")
    
    return output_path

def process_patch_directory(
    patch_dir: str,
    extractor,
    output_dir: str,
    batch_size: int = 32,
    file_pattern: str = "*.png",
    patch_size_level0: int = 1024
) -> str:
    """
    Process all patches in a directory and create H5 file.
    
    Args:
        patch_dir: Directory containing patch images
        extractor: ConchFeatureExtractor instance
        output_dir: Directory for output H5 file
        batch_size: Batch size for feature extraction
        file_pattern: File pattern to match (e.g., "*.png", "*.jpg")
        patch_size_level0: Patch size at level 0
        
    Returns:
        Path to created H5 file
    """
    # Get all image files
    image_paths = glob.glob(os.path.join(patch_dir, file_pattern))
    
    if not image_paths:
        raise ValueError(f"No images found in {patch_dir} with pattern {file_pattern}")
    
    # Sort paths for consistent ordering
    image_paths.sort()
    
    print(f"Found {len(image_paths)} images to process")
    
    # Extract features
    print("Extracting features...")
    features = extractor.extract_features(image_paths, batch_size=batch_size)
    
    # Create H5 file
    print("Creating H5 file...")
    return create_h5_from_features(
        features=features,
        image_paths=image_paths,
        output_dir=output_dir,
        patch_size_level0=patch_size_level0
    )

### Testing pipeline version 2

In [ ]:
# Example usage and testing
if __name__ == "__main__":
    # Test coordinate extraction
    # test_filenames = [
    #     "TCGA-AA-3846_nonMSIH_x3152_y22080_patch00015.png",
    #     "TCGA-PC-A5DK-01Z-00-DX1_x1024_y2048_patch00001.png",
    #     "Example_patch_00029_x2699_y27842.PNG"
    # ]
    
    # print("Testing coordinate extraction:")
    # for filename in test_filenames:
    #     try:
    #         x, y = extract_coords_from_filename(filename)
    #         print(f"{filename} -> x={x}, y={y}")
    #     except ValueError as e:
    #         print(f"Error with {filename}: {e}")
    
    # Example of creating H5 file
    extractor = ConchFeatureExtractor()
    
    # Process single directory
    h5_path = process_patch_directory(
        patch_dir="TCGA-A6-2671_nonMSIH",
        extractor=extractor,
        output_dir="output_dir/",
        batch_size=16
    )

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\datainsight\.cache\huggingface\token
Login successful
Found 710 images to process
Extracting features...
Creating H5 file...
H5 file created successfully: output_dir/TCGA-A6-2671_nonMSIH.h5
Features shape: (1, 710, 768)
Coords shape: (1, 710, 2)


In [16]:
inspect_h5_file("output_dir/TCGA-A6-2671_nonMSIH.h5")

Inspecting H5 file: output_dir/TCGA-A6-2671_nonMSIH.h5
Dataset: coords
  Shape: (1, 710, 2)
  Dtype: int64
  Attributes: {'contour_fn': 'four_pt', 'custom_downsample': 2.0, 'downsample': array([1., 1.]), 'name': 'TCGA-A6-2671_nonMSIH', 'patch_level': 0, 'patch_size': 1024, 'patch_size_level0': 1024, 'step_size': 1024, 'use_padding': True}

Dataset: features
  Shape: (1, 710, 768)
  Dtype: float32



### Pipeline version 3 with threading

In [2]:
import h5py
import numpy as np
import re
import os
from typing import List, Tuple, Dict, Any
from pathlib import Path
import glob
import time
from concurrent.futures import ThreadPoolExecutor
import torch
from PIL import Image

def extract_coords_from_filename(filename: str) -> Tuple[int, int]:
    """
    Extract x, y coordinates from patch filename.
    
    Expected format: TCGA-AA-3846_nonMSIH_x3152_y22080_patch00015.png
    
    Args:
        filename: Full filename or just the basename
        
    Returns:
        Tuple of (x, y) coordinates
        
    Raises:
        ValueError: If coordinates cannot be extracted from filename
    """
    # Remove path and extension to get just the basename
    basename = os.path.splitext(os.path.basename(filename))[0]
    
    # Pattern to match x and y coordinates
    pattern = r'_x(\d+)_y(\d+)'
    
    match = re.search(pattern, basename)
    if not match:
        raise ValueError(f"Could not extract coordinates from filename: {filename}")
    
    x_coord = int(match.group(1))
    y_coord = int(match.group(2))
    
    return x_coord, y_coord

def sort_by_coords(image_paths: List[str]) -> List[str]:
    def get_xy(path):
        try:
            return extract_coords_from_filename(path)
        except ValueError:
            return (float('inf'), float('inf'))  # Push invalid files to end
    return sorted(image_paths, key=get_xy)

def load_image_batch(image_paths: List[str], start_idx: int, batch_size: int, transform) -> torch.Tensor:
    """
    Load and transform a batch of images efficiently using threading.
    """
    end_idx = min(start_idx + batch_size, len(image_paths))
    batch_paths = image_paths[start_idx:end_idx]
    
    def load_single_image(path):
        try:
            img = Image.open(path).convert("RGB")
            return transform(img)
        except Exception as e:
            print(f"Warning: Failed to load {path}: {e}")
            return None
    
    # Use threading for I/O bound image loading
    with ThreadPoolExecutor(max_workers=min(8, len(batch_paths))) as executor:
        tensors = list(executor.map(load_single_image, batch_paths))
    
    # Filter out None values and stack
    valid_tensors = [t for t in tensors if t is not None]
    if not valid_tensors:
        return torch.empty(0)
    
    return torch.stack(valid_tensors)

def create_h5_from_features(
    features: np.ndarray,
    image_paths: List[str],
    output_dir: str,
    slide_name: str = None,
    patch_size_level0: int = 1024,
    additional_attributes: Dict[str, Any] = None
) -> str:
    """
    Create H5 file with features and coordinates extracted from image filenames.
    
    Args:
        features: Feature array of shape (n_patches, feature_dim)
        image_paths: List of image file paths corresponding to features
        output_dir: Directory where to save the H5 file
        slide_name: Name of the slide (extracted from first filename if None)
        patch_size_level0: Patch size at level 0
        additional_attributes: Additional attributes to add to coords dataset
        
    Returns:
        Path to created H5 file
    """
    if len(features) != len(image_paths):
        raise ValueError(f"Features length ({len(features)}) must match image_paths length ({len(image_paths)})")
    
    # Extract coordinates from filenames using threading for speed
    def extract_coords_worker(img_path):
        try:
            x, y = extract_coords_from_filename(img_path)
            return [x, y]
        except ValueError as e:
            print(f"Warning: {e}")
            return None
    
    # Parallel coordinate extraction
    with ThreadPoolExecutor(max_workers=8) as executor:
        coords_list = list(executor.map(extract_coords_worker, image_paths))
    
    # Filter out None values
    coords = np.array([c for c in coords_list if c is not None], dtype=np.int64)
    
    # Extract slide name and label from first filename for H5 filename
    if slide_name is None:
        first_filename = os.path.basename(image_paths[0])
        # Extract first 12 characters + label: TCGA-AA-3846_nonMSIH or TCGA-AA-3846_MSIH
        parts = first_filename.split('_')
        if len(parts) >= 2:
            slide_name = '_'.join(parts[:2])  # e.g., "TCGA-AA-3846_nonMSIH"
        else:
            slide_name = parts[0]
    
    # Create H5 filename from slide name
    h5_filename = f"{slide_name}_patch_features.h5"
    output_path = os.path.join(output_dir, h5_filename)
    
    # Ensure features have the right shape (add batch dimension if needed)
    if features.ndim == 2:
        features = features[np.newaxis, :]  # Add batch dimension: (1, n_patches, feature_dim)
    
    # Ensure coords have the right shape
    if coords.ndim == 2:
        coords = coords[np.newaxis, :]  # Add batch dimension: (1, n_patches, 2)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Create H5 file with optimized settings
    with h5py.File(output_path, 'w') as h5f:
        # Create features dataset with compression
        features_dataset = h5f.create_dataset(
            'features', 
            data=features.astype(np.float32),
            dtype=np.float32,
            compression='gzip',
            compression_opts=1,  # Light compression for speed
            chunks=True
        )
        
        # Create coordinates dataset
        coords_dataset = h5f.create_dataset(
            'coords',
            data=coords.astype(np.int64),
            dtype=np.int64,
            compression='gzip',
            compression_opts=1,
            chunks=True
        )
        
        # Add attributes to coords dataset
        default_attributes = {
            'contour_fn': 'four_pt',
            'custom_downsample': 2.0,
            'downsample': np.array([1., 1.]),
            'name': slide_name,
            'patch_level': 0,
            'patch_size': patch_size_level0,
            'patch_size_level0': patch_size_level0,
            'step_size': patch_size_level0,
            'use_padding': True
        }
        
        # Add additional attributes if provided
        if additional_attributes:
            default_attributes.update(additional_attributes)
        
        # Set attributes
        for key, value in default_attributes.items():
            coords_dataset.attrs[key] = value
    
    print(f"H5 file created successfully: {output_path}")
    print(f"Features shape: {features.shape}")
    print(f"Coords shape: {coords.shape}")
    
    return output_path

def process_patch_directory_optimized(
    patch_dir: str,
    extractor,
    output_dir: str,
    csv_path: str,
    batch_size: int = 64,  # Increased default batch size
    file_pattern: str = "*.png",
    patch_size_level0: int = 1024,
    max_workers: int = 8,
    progress_interval: int = 10
) -> str:
    """
    Process all patches in a directory and create H5 file with optimizations.
    
    Args:
        patch_dir: Directory containing patch images
        extractor: ConchFeatureExtractor instance
        output_dir: Directory for output H5 file
        batch_size: Batch size for feature extraction (larger = faster)
        file_pattern: File pattern to match (e.g., "*.png", "*.jpg")
        patch_size_level0: Patch size at level 0
        max_workers: Number of threads for image loading
        progress_interval: Print progress every N batches
        
    Returns:
        Path to created H5 file
    """
    # Extract slide name from directory name
    slide_name = os.path.basename(patch_dir.rstrip('/\\'))
    
    # Load Excel and find matching patch size
    df_meta = pd.read_csv(csv_path)
    
    # Find row where "renamed" matches slide_name (case-insensitive match)
    row = df_meta[df_meta['renamed'].str.lower() == slide_name.lower()]
    
    if row.empty:
        raise ValueError(f"Slide name '{slide_name}' not found in csv file.")
    
    objective_power = int(row.iloc[0]['Objective Power'])
    if objective_power == 40:
        patch_size_level0 = 1024
    elif objective_power == 20:
        patch_size_level0 = 512
    else:
        raise ValueError(f"Unsupported objective power: {objective_power} for slide {slide_name}")
    
    print(f"Detected patch size: {patch_size_level0} for slide '{slide_name}' with objective power {objective_power}")

    start_time = time.time()
    
    # Get all image files
    print("Scanning for images...")
    image_paths = glob.glob(os.path.join(patch_dir, file_pattern))
    
    if not image_paths:
        raise ValueError(f"No images found in {patch_dir} with pattern {file_pattern}")
    
    # Sort paths for consistent ordering
    image_paths = sort_by_coords(image_paths)
    
    print(f"Found {len(image_paths)} images to process")
    print(f"Using batch size: {batch_size}")
    print(f"Estimated batches: {len(image_paths) // batch_size + 1}")
    
    # Pre-allocate result arrays for better memory efficiency
    n_images = len(image_paths)
    feature_dim = None
    all_features = []
    
    # Process in batches with optimized loading
    print("Extracting features...")
    batch_count = 0
    
    for i in range(0, n_images, batch_size):
        batch_start = time.time()
        
        # Load batch of images with threading
        batch_tensor = load_image_batch(image_paths, i, batch_size, extractor.eval_transform)
        
        if batch_tensor.numel() == 0:
            print(f"Warning: Empty batch at index {i}")
            continue
        
        # Move to GPU and extract features
        batch_tensor = batch_tensor.to(extractor.device)
        
        with torch.no_grad():
            features = extractor.conch(batch_tensor)
            features_np = features.cpu().numpy()
            all_features.append(features_np)
        
        batch_count += 1
        
        # Progress reporting
        if batch_count % progress_interval == 0:
            batch_time = time.time() - batch_start
            processed = min(i + batch_size, n_images)
            elapsed = time.time() - start_time
            rate = processed / elapsed
            eta = (n_images - processed) / rate if rate > 0 else 0
            
            print(f"Batch {batch_count}: {processed}/{n_images} images "
                  f"({processed/n_images*100:.1f}%) - "
                  f"Rate: {rate:.1f} imgs/sec - "
                  f"ETA: {eta:.1f}s - "
                  f"Batch time: {batch_time:.2f}s")
    
    # Concatenate all features
    if not all_features:
        raise ValueError("No features extracted - check your images")
    
    features = np.concatenate(all_features, axis=0)
    
    extraction_time = time.time() - start_time
    print(f"Feature extraction completed in {extraction_time:.2f}s")
    print(f"Average rate: {len(image_paths)/extraction_time:.1f} images/sec")
    
    # Create H5 file
    print("Creating H5 file...")
    h5_start = time.time()
    
    h5_path = create_h5_from_features(
        features=features,
        image_paths=image_paths,
        output_dir=output_dir,
        patch_size_level0=patch_size_level0
    )
    
    h5_time = time.time() - h5_start
    total_time = time.time() - start_time
    
    print(f"H5 creation completed in {h5_time:.2f}s")
    print(f"Total processing time: {total_time:.2f}s")
    print(f"Overall rate: {len(image_paths)/total_time:.1f} images/sec")
    
    return h5_path

# Performance benchmarking function
def benchmark_processing(
    patch_dir: str,
    extractor,
    output_dir: str,
    batch_sizes: List[int] = [32, 64, 128, 256]
) -> Dict[int, float]:
    """
    Benchmark different batch sizes to find optimal performance.
    
    Args:
        patch_dir: Directory containing patch images
        extractor: ConchFeatureExtractor instance
        output_dir: Directory for output H5 file
        batch_sizes: List of batch sizes to test
        
    Returns:
        Dictionary mapping batch_size to processing time
    """
    results = {}
    
    # Get image count
    image_paths = glob.glob(os.path.join(patch_dir, "*.png"))
    n_images = len(image_paths)
    
    print(f"Benchmarking with {n_images} images")
    
    for batch_size in batch_sizes:
        print(f"\nTesting batch size: {batch_size}")
        
        try:
            start_time = time.time()
            
            # Just test feature extraction (skip H5 creation for speed)
            for i in range(0, min(n_images, 500), batch_size):  # Test first 500 images
                batch_tensor = load_image_batch(image_paths, i, batch_size, extractor.eval_transform)
                
                if batch_tensor.numel() == 0:
                    continue
                
                batch_tensor = batch_tensor.to(extractor.device)
                
                with torch.no_grad():
                    features = extractor.conch(batch_tensor)
                    _ = features.cpu().numpy()
            
            elapsed = time.time() - start_time
            rate = min(500, n_images) / elapsed
            
            results[batch_size] = rate
            print(f"Batch size {batch_size}: {rate:.1f} images/sec")
            
        except Exception as e:
            print(f"Batch size {batch_size} failed: {e}")
            results[batch_size] = 0
    
    return results

In [ ]:
# Benchmark different batch sizes
if __name__ == "__main__":
    # Example benchmarking
    extractor = ConchFeatureExtractor()
    
    # Find optimal batch size
    results = benchmark_processing(
        patch_dir="TCGA-A6-2671_nonMSIH",
        extractor=extractor,
        output_dir="output_dir/",
        batch_sizes=[32, 64, 128, 256]
    )
    
    optimal_batch_size = max(results.keys(), key=lambda k: results[k])
    print(f"Optimal batch size: {optimal_batch_size}")

In [60]:
import pandas as pd
# Process with optimal settings
h5_path = process_patch_directory_optimized(
    patch_dir="TCGA-A6-2671_nonMSIH",
    extractor=extractor,
    output_dir="output_dir/",
    csv_path="slide_metadata_filtered.csv",
    batch_size=optimal_batch_size,
    max_workers=8,
    progress_interval=5
)

Detected patch size: 1024 for slide 'TCGA-A6-2671_nonMSIH' with objective power 40
Scanning for images...
Found 710 images to process
Using batch size: 64
Estimated batches: 12
Extracting features...
Batch 5: 320/710 images (45.1%) - Rate: 50.4 imgs/sec - ETA: 7.7s - Batch time: 1.21s
Batch 10: 640/710 images (90.1%) - Rate: 51.4 imgs/sec - ETA: 1.4s - Batch time: 1.22s
Feature extraction completed in 13.78s
Average rate: 51.5 images/sec
Creating H5 file...
H5 file created successfully: output_dir/TCGA-A6-2671_nonMSIH_patch_features.h5
Features shape: (1, 710, 768)
Coords shape: (1, 710, 2)
H5 creation completed in 0.06s
Total processing time: 13.84s
Overall rate: 51.3 images/sec


### Test to make sure h5 file correct

In [38]:
# Path to H5 file from which we check first 5
h5_path = "output_dir/TCGA-A6-2671_nonMSIH.h5"  # change this to your file path

# Open the file and inspect
with h5py.File(h5_path, 'r') as h5f:
    features = h5f['features'][0][:5]  # shape: (n_patches, feature_dim) — take first 5
    coords = h5f['coords'][0][:5]      # shape: (n_patches, 2) — take first 5

    print("First 5 Coordinates:")
    print(coords)

    print("\nFirst 5 Feature Vectors (truncated to 10 dims for display):")
    for i, feat in enumerate(features):
        print(f"Patch {i+1}: {feat[:10]}...")  # Truncate to first 10 elements for readability


First 5 Coordinates:
[[ 592 9200]
 [ 592 9712]
 [1104 2544]
 [1104 3056]
 [1104 3568]]

First 5 Feature Vectors (truncated to 10 dims for display):
Patch 1: [-0.64296025  0.71390176 -2.523942   -0.9881429   0.01601001  0.9952705
  1.0412811  -0.34713998  0.01437233 -1.8884089 ]...
Patch 2: [-0.7682956   0.8862452  -2.8391075   0.01998501 -0.20540938  0.15393399
  1.4437208   0.1595187   0.02632063 -1.4302748 ]...
Patch 3: [ 0.33677113 -0.17703715 -3.3091826  -0.42305347 -0.9021348  -0.25092685
  2.0151381  -0.476161   -0.10059924 -1.1529248 ]...
Patch 4: [-0.6808688   0.4103926  -2.5359395  -1.7148114   0.44247442 -0.13393672
  1.079024   -0.5313186   0.8069699  -1.0319271 ]...
Patch 5: [-0.32306755  0.51347786 -2.6778853  -1.1545395   0.585494    0.38392475
  1.0787324  -0.10163652  0.5530322  -1.3799305 ]...


In [32]:
# Path to H5 file of just first 5 images
h5_path = "output_dir/TCGA-A6-2671_nonMSIH.h5"  # change this to your file path

# Open the file and inspect
with h5py.File(h5_path, 'r') as h5f:
    features = h5f['features'][0][:5]  # shape: (n_patches, feature_dim) — take first 5
    coords = h5f['coords'][0][:5]      # shape: (n_patches, 2) — take first 5

    print("First 5 Coordinates:")
    print(coords)

    print("\nFirst 5 Feature Vectors (truncated to 10 dims for display):")
    for i, feat in enumerate(features):
        print(f"Patch {i+1}: {feat[:10]}...")  # Truncate to first 10 elements for readability

First 5 Coordinates:
[[10320  2544]
 [10320  3056]
 [10320  3568]
 [10320  4080]
 [10320  4592]]

First 5 Feature Vectors (truncated to 10 dims for display):
Patch 1: [-0.84444135  1.1016262  -2.7733774   0.8028838  -0.8070753   0.74785304
  0.9708581  -0.670074   -0.3604539  -1.5672109 ]...
Patch 2: [-0.2721286   0.815057   -2.889659    0.21067041 -0.38861454  0.39771283
  0.9664094  -0.00370616 -0.14306515 -2.1560702 ]...
Patch 3: [ 0.40740287 -0.02443755 -2.538565   -0.3824261  -0.09485035  0.07231339
  0.9076131   0.22004066  0.20590293 -0.8040727 ]...
Patch 4: [-0.29466745  0.85380936 -3.1747298   0.19728047 -0.76919615 -0.10895678
  1.1735804  -0.0879968   0.22204544 -0.8563476 ]...
Patch 5: [-0.5267402   1.1048594  -2.9155889   0.40441254 -0.2957316  -0.04036119
  0.5495645   0.04806878  0.31227407 -0.6960502 ]...


In [48]:
# Path to H5 file from which we check first N
h5_path = "output_dir/TCGA-A6-2671_nonMSIH.h5"  # change this to your file path
n = 100
# Open the file and inspect
with h5py.File(h5_path, 'r') as h5f:
    # features = h5f['features'][0][:5]  # shape: (n_patches, feature_dim) — take first 5
    coords = h5f['coords'][0][:n]      # shape: (n_patches, 2) — take first 5

    print("First " + str(n) + " Coordinates:")
    print(coords)

    # print("\nFirst 5 Feature Vectors (truncated to 10 dims for display):")
    # for i, feat in enumerate(features):
    #     print(f"Patch {i+1}: {feat[:10]}...")  # Truncate to first 10 elements for readability

First 100 Coordinates:
[[  592  9200]
 [  592  9712]
 [ 1104  2544]
 [ 1104  3056]
 [ 1104  3568]
 [ 1104  4080]
 [ 1104  4592]
 [ 1104  5104]
 [ 1104  5616]
 [ 1104  6128]
 [ 1104  6640]
 [ 1104  8688]
 [ 1104  9200]
 [ 1104  9712]
 [ 1104 10224]
 [ 1616  2544]
 [ 1616  3056]
 [ 1616  3568]
 [ 1616  4080]
 [ 1616  4592]
 [ 1616  5104]
 [ 1616  5616]
 [ 1616  6128]
 [ 1616  6640]
 [ 1616  7152]
 [ 1616  7664]
 [ 1616  8176]
 [ 1616  8688]
 [ 1616  9200]
 [ 1616  9712]
 [ 1616 10224]
 [ 1616 10736]
 [ 2128  2544]
 [ 2128  3056]
 [ 2128  3568]
 [ 2128  4080]
 [ 2128  4592]
 [ 2128  5104]
 [ 2128  5616]
 [ 2128  6128]
 [ 2128  6640]
 [ 2128  7152]
 [ 2128  7664]
 [ 2128  8176]
 [ 2128  8688]
 [ 2128  9200]
 [ 2128  9712]
 [ 2128 10224]
 [ 2128 10736]
 [ 2640  2032]
 [ 2640  2544]
 [ 2640  3056]
 [ 2640  3568]
 [ 2640  4080]
 [ 2640  4592]
 [ 2640  5104]
 [ 2640  5616]
 [ 2640  6128]
 [ 2640  6640]
 [ 2640  7152]
 [ 2640  7664]
 [ 2640  8176]
 [ 2640  8688]
 [ 2640  9200]
 [ 2640  9712]
 [

## Extract Slide-level Embedding

In [49]:
# Load model
titan = AutoModel.from_pretrained('MahmoodLab/TITAN', trust_remote_code=True)

# Path to H5 file
h5_path = 'output_dir/TCGA-A6-2671_nonMSIH_patch_features.h5'

# Extract slide name (remove directory + "_patch_features.h5")
slide_name = os.path.basename(h5_path).replace('_patch_features.h5', '')

# Load features and metadata
with h5py.File(h5_path, 'r') as file:
    features = torch.from_numpy(file['features'][:])
    coords = torch.from_numpy(file['coords'][:])
    patch_size_lv0 = file['coords'].attrs['patch_size_level0']

# Extract slide embedding
with torch.autocast('cuda', torch.float16), torch.inference_mode():
    slide_embedding = titan.encode_slide_from_patch_features(features, coords, patch_size_lv0)

# Save embedding with correct name
slide_embedding_name = os.path.join(os.path.dirname(h5_path), f"{slide_name}_slide_embedding.pt")
torch.save(slide_embedding, slide_embedding_name)

print(f"Slide embedding saved to: {slide_embedding_name}")

Slide embedding saved to: output_dir\TCGA-A6-2671_nonMSIH_slide_embedding.pt


In [44]:
print(slide_embedding.shape)

torch.Size([1, 768])


### Script to skip white patches specified by csv

In [ ]:
import os
import glob
import pandas as pd

# Load non-white patch names from CSV
non_white_df = pd.read_csv("final_merged_without_white.csv")  # has column: patch_name
valid_patch_names = set(non_white_df['patch_name'].astype(str))  # Use set for O(1) lookups

# Directory containing patches for a specific WSI
patch_dir = "TCGA-AA-3846_nonMSIH"

# Initialize counters
skipped_patches = 0
unskipped_patches = 0

# Get all patch image paths
all_patch_paths = glob.glob(os.path.join(patch_dir, "*.png"))

# Process each patch
for patch_path in all_patch_paths:
    patch_name = os.path.basename(patch_path)

    if patch_name not in valid_patch_names:
        skipped_patches += 1
        print("Skipped patch:", patch_name)
        continue

    unskipped_patches += 1
    print("Processing patch:", patch_name)

# Summary
print("Total processed patches:", unskipped_patches)
print("Total skipped patches:", skipped_patches)


Skipped patch: TCGA-AA-3846_nonMSIH_x10320_y13888_patch00457.png
Skipped patch: TCGA-AA-3846_nonMSIH_x10320_y14400_patch00458.png
Processing patch: TCGA-AA-3846_nonMSIH_x10320_y14912_patch00459.png
Processing patch: TCGA-AA-3846_nonMSIH_x10320_y15424_patch00460.png
Processing patch: TCGA-AA-3846_nonMSIH_x10320_y15936_patch00461.png
Processing patch: TCGA-AA-3846_nonMSIH_x10320_y16448_patch00462.png
Processing patch: TCGA-AA-3846_nonMSIH_x10320_y16960_patch00463.png
Processing patch: TCGA-AA-3846_nonMSIH_x10320_y17472_patch00464.png
Processing patch: TCGA-AA-3846_nonMSIH_x10320_y17984_patch00465.png
Processing patch: TCGA-AA-3846_nonMSIH_x10320_y18496_patch00466.png
Processing patch: TCGA-AA-3846_nonMSIH_x10320_y19008_patch00467.png
Processing patch: TCGA-AA-3846_nonMSIH_x10320_y19520_patch00468.png
Processing patch: TCGA-AA-3846_nonMSIH_x10320_y20032_patch00469.png
Processing patch: TCGA-AA-3846_nonMSIH_x10320_y20544_patch00470.png
Processing patch: TCGA-AA-3846_nonMSIH_x10320_y21056_p

## Final Pipeline

###### Access D:\Aamir Gulzar\KSA_project2\dataset\patch_data
###### Iterate through all subfolders and which all have names like TCGA-3L-AA1B_nonMSIH and extract patch level features from them
###### Skip the patches whose name is not in final_merged_without_white.csv
###### patch features are saved in h5, save that h5 in a folder in D:\Aamir Gulzar\KSA_project2\dataset\features\TCGA-3L-AA1B_nonMSIH\patch_features
###### h5 should have name like TCGA-3L-AA1B_nonMSIH_patch_features
###### Extract slide embedding and save that also in D:\Aamir Gulzar\KSA_project2\dataset\features\TCGA-3L-AA1B_nonMSIH\slide_features

In [4]:
import os
import glob
import pandas as pd
import torch
from h5py import File as H5File
from transformers import AutoModel

def run_full_pipeline(
    patch_data_dir: str,
    extractor,
    slide_meta_csv: str,
    non_white_csv: str,
    output_root: str,
    batch_size: int = 64
):
    # Load slide metadata and non-white patch names
    slide_meta_df = pd.read_csv(slide_meta_csv)
    non_white_df = pd.read_csv(non_white_csv)
    valid_patch_names = set(non_white_df['patch_name'].astype(str))

    # Load TITAN model
    titan = AutoModel.from_pretrained("MahmoodLab/TITAN", trust_remote_code=True)

    # Iterate through all slide folders in patch_data_dir
    slide_dirs = glob.glob(os.path.join(patch_data_dir, "*"))

    for slide_dir in slide_dirs:
        slide_name = os.path.basename(slide_dir)
        print(f"\nProcessing slide: {slide_name}")

        # Match objective power
        meta_row = slide_meta_df[slide_meta_df["renamed"].str.lower() == slide_name.lower()]
        if meta_row.empty:
            print(f"  ❌ Skipped: Slide metadata not found for {slide_name}")
            continue

        try:
            obj_power = int(meta_row.iloc[0]["Objective Power"])
            patch_size_level0 = 1024 if obj_power == 40 else 512 if obj_power == 20 else None
            if patch_size_level0 is None:
                print(f"  ❌ Unsupported objective power: {obj_power}")
                continue
        except Exception as e:
            print(f"  ❌ Error extracting objective power: {e}")
            continue

        # Filter valid patches for this slide
        all_patches = glob.glob(os.path.join(slide_dir, "*.png"))
        patch_paths = [p for p in all_patches if os.path.basename(p) in valid_patch_names]

        if not patch_paths:
            print(f"  ❌ No valid non-white patches found for {slide_name}")
            continue

        # Sort and extract features
        patch_paths = sort_by_coords(patch_paths)
        output_dir = os.path.join(output_root, slide_name)
        os.makedirs(output_dir, exist_ok=True)

        features_list = []
        for i in range(0, len(patch_paths), batch_size):
            batch = load_image_batch(patch_paths, i, batch_size, extractor.eval_transform)
            if batch.numel() == 0:
                continue
            batch = batch.to(extractor.device)
            with torch.no_grad():
                feats = extractor.conch(batch).cpu().numpy()
                features_list.append(feats)

        if not features_list:
            print(f"  ❌ No features extracted for {slide_name}")
            continue

        features_np = np.concatenate(features_list, axis=0)
        h5_path = create_h5_from_features(
            features_np,
            patch_paths,
            output_dir=os.path.join(output_dir, "patch_features"),
            slide_name=slide_name,
            patch_size_level0=patch_size_level0
        )

        # Save a .pt version of the h5 contents
        with H5File(h5_path, 'r') as file:
            patch_features = torch.from_numpy(file['features'][:])
            patch_coords = torch.from_numpy(file['coords'][:])
            patch_size_lv0 = file['coords'].attrs['patch_size_level0']
        
        pt_data = {
            'features': patch_features,
            'coords': patch_coords,
            'patch_size_level0': patch_size_lv0
        }
        
        pt_path = h5_path.replace('.h5', '.pt')
        torch.save(pt_data, pt_path)
        
        print(f"  💾 Saved patch data to .pt at {pt_path}")

        # Extract slide embedding
        with H5File(h5_path, 'r') as file:
            feats = torch.from_numpy(file['features'][:])
            coords = torch.from_numpy(file['coords'][:])
            patch_size_lv0 = file['coords'].attrs['patch_size_level0']

        with torch.autocast('cuda', torch.float16), torch.inference_mode():
            slide_emb = titan.encode_slide_from_patch_features(feats, coords, patch_size_lv0)

        slide_emb_path = os.path.join(output_dir, "slide_features", f"{slide_name}_slide_embedding.pt")
        os.makedirs(os.path.dirname(slide_emb_path), exist_ok=True)
        torch.save(slide_emb, slide_emb_path)

        print(f"  ✅ Done: Saved H5 and slide embedding for {slide_name}")

In [ ]:
extractor = ConchFeatureExtractor()

run_full_pipeline(
    patch_data_dir=r"D:\Aamir Gulzar\KSA_project2\dataset\Test_data",
    extractor=extractor,
    slide_meta_csv=r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\slide_metadata_filtered.csv",
    non_white_csv=r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\final_merged_without_white.csv",
    output_root=r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5",
    batch_size=64
)

### Test version of final pipeline

In [15]:
# Test version of final pipeline with print statements
import os
import glob
import pandas as pd
import torch
import numpy as np
from h5py import File as H5File
from transformers import AutoModel

def run_full_pipeline_test(
    patch_data_dir: str,
    extractor,
    slide_meta_csv: str,
    non_white_csv: str,
    output_root: str,
    batch_size: int = 64
):
    # Load slide metadata and non-white patch names
    print("🔄 Loading metadata CSVs...")
    slide_meta_df = pd.read_csv(slide_meta_csv)
    non_white_df = pd.read_csv(non_white_csv)
    valid_patch_names = set(non_white_df['patch_name'].astype(str))
    print(f"✅ Loaded {len(slide_meta_df)} slide metadata entries.")
    print(f"✅ Loaded {len(valid_patch_names)} non-white patch names.")

    # Load TITAN model
    print("🔄 Loading TITAN model...")
    titan = AutoModel.from_pretrained("MahmoodLab/TITAN", trust_remote_code=True)
    print("✅ TITAN model loaded.")

    # Iterate through all slide folders in patch_data_dir
    slide_dirs = glob.glob(os.path.join(patch_data_dir, "*"))
    print(f"📁 Found {len(slide_dirs)} slide directories.")

    for slide_dir in slide_dirs:
        slide_name = os.path.basename(slide_dir)
        print(f"\n🚩 Processing slide: {slide_name}")
        print(f"🔍 Looking up metadata for slide...")

        # Match objective power
        meta_row = slide_meta_df[slide_meta_df["renamed"].str.lower() == slide_name.lower()]
        if meta_row.empty:
            print(f"  ❌ Skipped: Slide metadata not found for {slide_name}")
            continue

        try:
            obj_power = int(meta_row.iloc[0]["Objective Power"])
            patch_size_level0 = 1024 if obj_power == 40 else 512 if obj_power == 20 else None
            print(f"🔬 Objective Power: {obj_power}, Patch Size LV0: {patch_size_level0}")
            if patch_size_level0 is None:
                print(f"  ❌ Unsupported objective power: {obj_power}")
                continue
        except Exception as e:
            print(f"  ❌ Error extracting objective power: {e}")
            continue

        # Filter valid patches for this slide
        all_patches = glob.glob(os.path.join(slide_dir, "*.png"))
        print(f"🧩 Found {len(all_patches)} total patches.")
        patch_paths = [p for p in all_patches if os.path.basename(p) in valid_patch_names]
        print(f"✅ {len(patch_paths)} non-white patches retained.")

        if not patch_paths:
            print(f"  ❌ No valid non-white patches found for {slide_name}")
            continue

        # Sort and extract features
        print("📏 Sorting patch paths by coordinates...")
        patch_paths = sort_by_coords(patch_paths)
        output_dir = os.path.join(output_root, slide_name)
        os.makedirs(output_dir, exist_ok=True)

        print("🧠 Starting feature extraction...")
        features_list = []
        for i in range(0, len(patch_paths), batch_size):
            print(f"  🔄 Loading batch {i // batch_size + 1} of {len(patch_paths) // batch_size + 1}")
            batch = load_image_batch(patch_paths, i, batch_size, extractor.eval_transform)
            print(f"  🔢 Batch shape: {batch.shape}")

            if batch.numel() == 0:
                print("  ⚠️ Empty batch, skipping...")
                continue
            batch = batch.to(extractor.device)
            with torch.no_grad():
                feats = extractor.conch(batch).cpu().numpy()
                print(f"  ✅ Extracted features of shape: {feats.shape}")
                features_list.append(feats)

        if not features_list:
            print(f"  ❌ No features extracted for {slide_name}")
            continue

        features_np = np.concatenate(features_list, axis=0)
        print(f"📦 Total feature array shape: {features_np.shape}")

        h5_path = create_h5_from_features(
            features_np,
            patch_paths,
            output_dir=os.path.join(output_dir, "patch_features"),
            slide_name=slide_name,
            patch_size_level0=patch_size_level0
        )
        print(f"💾 Saved features to H5: {h5_path}")

        # Save a .pt version of the h5 contents
        with H5File(h5_path, 'r') as file:
            patch_features = torch.from_numpy(file['features'][:])
            patch_coords = torch.from_numpy(file['coords'][:])
            patch_size_lv0 = file['coords'].attrs['patch_size_level0']
        
        pt_data = {
            'features': patch_features,
            'coords': patch_coords,
            'patch_size_level0': patch_size_lv0
        }
        
        pt_path = h5_path.replace('.h5', '.pt')
        torch.save(pt_data, pt_path)
        
        print(f"  💾 Saved patch data to .pt at {pt_path}")

        # Extract slide embedding
        print("🧠 Extracting slide embedding from TITAN...")
        with H5File(h5_path, 'r') as file:
            feats = torch.from_numpy(file['features'][:])
            coords = torch.from_numpy(file['coords'][:])
            patch_size_lv0 = file['coords'].attrs['patch_size_level0']
            print(f"  🔍 Read {feats.shape[0]} features with patch size {patch_size_lv0}")

        with torch.autocast('cuda', torch.float16), torch.inference_mode():
            slide_emb = titan.encode_slide_from_patch_features(feats, coords, patch_size_lv0)
            print(f"  ✅ Slide embedding shape: {slide_emb.shape}")

        slide_emb_path = os.path.join(output_dir, "slide_features", f"{slide_name}_slide_embedding.pt")
        os.makedirs(os.path.dirname(slide_emb_path), exist_ok=True)
        torch.save(slide_emb, slide_emb_path)
        print(f"📁 Saved slide embedding to {slide_emb_path}")
        print(f"✅ Done with {slide_name}")

In [ ]:
extractor = ConchFeatureExtractor()

# For PC testing
run_full_pipeline_test(
    patch_data_dir=r"D:\Aamir Gulzar\KSA_project2\dataset\Test_data",
    extractor=extractor,
    slide_meta_csv=r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\slide_metadata_filtered.csv",
    non_white_csv=r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\final_merged_without_white.csv",
    output_root=r"D:\Aamir Gulzar\KSA_project2\dataset\Features_test",
    batch_size=64
)

# # For Laptop testing
# run_full_pipeline_test(
#     patch_data_dir=r"D:\Titan_Project\Test_data_smaller",
#     extractor=extractor,
#     slide_meta_csv=r"D:\Titan_Project\slide_metadata_filtered.csv",
#     non_white_csv=r"D:\Titan_Project\final_merged_without_white.csv",
#     output_root=r"D:\Titan_Project\output_dir",
#     batch_size=64
# )

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\Hp\.cache\huggingface\token
Login successful
🔄 Loading metadata CSVs...
✅ Loaded 425 slide metadata entries.
✅ Loaded 1377933 non-white patch names.
🔄 Loading TITAN model...
✅ TITAN model loaded.
📁 Found 3 slide directories.

🚩 Processing slide: TCGA-3L-AA1B_nonMSIH
🔍 Looking up metadata for slide...
🔬 Objective Power: 40, Patch Size LV0: 1024
🧩 Found 5 total patches.
✅ 5 non-white patches retained.
📏 Sorting patch paths by coordinates...
🧠 Starting feature extraction...
  🔄 Loading batch 1 of 1
  🔢 Batch shape: torch.Size([5, 3, 448, 448])
  ✅ Extracted features of shape: (5, 768)
📦 Total feature array shape: (5, 768)
H5 file created successfully: D:\Titan_Project\output_dir\TCGA-3L-AA1B_nonMSIH\patch_features\TCGA-3L-AA1B_nonMSIH_patch_features.h5
Features shape: (1, 5, 768)

d:\miniconda_3\envs\titan_env\Lib\site-packages\torch\amp\autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


  ✅ Slide embedding shape: torch.Size([1, 768])
📁 Saved slide embedding to D:\Titan_Project\output_dir\TCGA-3L-AA1B_nonMSIH\slide_features\TCGA-3L-AA1B_nonMSIH_slide_embedding.pt
✅ Done with TCGA-3L-AA1B_nonMSIH

🚩 Processing slide: TCGA-A6-5661_MSIH
🔍 Looking up metadata for slide...
🔬 Objective Power: 40, Patch Size LV0: 1024
🧩 Found 5 total patches.
✅ 5 non-white patches retained.
📏 Sorting patch paths by coordinates...
🧠 Starting feature extraction...
  🔄 Loading batch 1 of 1
  🔢 Batch shape: torch.Size([5, 3, 448, 448])
  ✅ Extracted features of shape: (5, 768)
📦 Total feature array shape: (5, 768)
H5 file created successfully: D:\Titan_Project\output_dir\TCGA-A6-5661_MSIH\patch_features\TCGA-A6-5661_MSIH_patch_features.h5
Features shape: (1, 5, 768)
Coords shape: (1, 5, 2)
💾 Saved features to H5: D:\Titan_Project\output_dir\TCGA-A6-5661_MSIH\patch_features\TCGA-A6-5661_MSIH_patch_features.h5
  💾 Saved patch data to .pt at D:\Titan_Project\output_dir\TCGA-A6-5661_MSIH\patch_featur

In [9]:
h5_path = r"D:\Aamir Gulzar\KSA_project2\dataset\Features\TCGA-3L-AA1B_nonMSIH\patch_features\TCGA-3L-AA1B_nonMSIH_patch_features.h5"

# Open and read the features
with h5py.File(h5_path, 'r') as h5_file:
    features = h5_file['features'][:]  # shape: (N_patches, feature_dim)

print(features.shape)

features = features[0]  # shape becomes (1878, 768)

# Compute variance across all patches for each feature dimension
variances = np.var(features, axis=0)

# Compute and print mean variance
mean_variance = np.mean(variances)
print("Mean variance of patch features:", mean_variance)

(1, 1878, 768)
Mean variance of patch features: 0.42321482


## Pipeline for feature extraction using UNI_2h

In [6]:
import torch
from PIL import Image
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

class UNI2Extractor:
    def __init__(self, device='cuda'):
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')

        timm_kwargs = {
            'img_size': 224,
            'patch_size': 14,
            'depth': 24,
            'num_heads': 24,
            'init_values': 1e-5,
            'embed_dim': 1536,
            'mlp_ratio': 2.66667 * 2,
            'num_classes': 0,
            'no_embed_class': True,
            'mlp_layer': timm.layers.SwiGLUPacked,
            'act_layer': torch.nn.SiLU,
            'reg_tokens': 8,
            'dynamic_img_size': True
        }

        self.model = timm.create_model("hf-hub:MahmoodLab/UNI2-h", pretrained=True, **timm_kwargs).to(self.device)
        self.model.eval()

        config = resolve_data_config(self.model.pretrained_cfg, model=self.model)
        self.eval_transform = create_transform(**config)

    def extract_features(self, batch_imgs):
        # batch_imgs: tensor of shape (B, 3, 224, 224)
        with torch.no_grad():
            return self.model(batch_imgs)

### Uni2-h feature extrator fivecrop

In [16]:
import torch
from PIL import Image
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from torchvision import transforms

class UNI2ExtractorFiveCrop():
    def __init__(self, device='cuda'):
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')

        timm_kwargs = {
            'img_size': 224,
            'patch_size': 14,
            'depth': 24,
            'num_heads': 24,
            'init_values': 1e-5,
            'embed_dim': 1536,
            'mlp_ratio': 2.66667 * 2,
            'num_classes': 0,
            'no_embed_class': True,
            'mlp_layer': timm.layers.SwiGLUPacked,
            'act_layer': torch.nn.SiLU,
            'reg_tokens': 8,
            'dynamic_img_size': True
        }

        self.model = timm.create_model("hf-hub:MahmoodLab/UNI2-h", pretrained=True, **timm_kwargs).to(self.device)
        self.model.eval()

        # Get base transform for individual crops
        config = resolve_data_config(self.model.pretrained_cfg, model=self.model)
        self.eval_transform = create_transform(**config)

        # FiveCrop transform: crop → transform each crop → stack into 5x3x224x224
        self.fivecrop_transform = transforms.Compose([
            transforms.FiveCrop(224),  # five 224x224 crops from the 512x512 image
            transforms.Lambda(lambda crops: torch.stack([self.eval_transform(c) for c in crops]))
        ])

    def extract_fivecrop_features(self, image: Image.Image | str) -> torch.Tensor:
        """
        Extract 5xD features from one image using FiveCrop.
        Returns: Tensor of shape (5, D)
        """
        if isinstance(image, str):
            image = Image.open(image).convert("RGB")
        else:
            image = image.convert("RGB")

        crops_tensor = self.fivecrop_transform(image).to(self.device)  # shape: (5, 3, 224, 224)

        with torch.no_grad():
            features = self.model(crops_tensor)  # shape: (5, D)

        return features.cpu()
    

    def extract_batch_features(self, image_paths: List[str]) -> List[tuple[str, torch.Tensor]]:
        tensors = []
        valid_paths = []

        for path in image_paths:
            try:
                img = Image.open(path).convert("RGB")
                crops = self.fivecrop_transform(img)  # shape: (5, 3, 224, 224)
                tensors.append(crops)
                valid_paths.append(path)
            except Exception as e:
                print(f"Skipping {path}: {e}")

        if not tensors:
            return []

        batch_tensor = torch.cat(tensors, dim=0).to(self.device)  # (N*5, 3, 224, 224)

        with torch.no_grad():
            feats = self.model(batch_tensor)  # (N*5, D)

        D = feats.shape[1]
        feats = feats.view(len(valid_paths), 5, D).cpu()  # (N, 5, D)

        return list(zip(valid_paths, feats))  # [(filepath, 5xD tensor), ...]


### Testing

In [20]:
# === MAIN SCRIPT ===

patch_root = r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\test_data"
save_h5_path = r"D:\Aamir Gulzar\KSA_project2\dataset\Features_test\Uni_2h_fivecrop.h5"
batch_size = 64

# Gather patch image paths
patch_paths = glob.glob(os.path.join(patch_root, "*", "*.png"))
extractor = UNI2ExtractorFiveCrop()

all_coords = []
all_features = []

# Batched feature extraction
for i in tqdm(range(0, len(patch_paths), batch_size), desc="Extracting features"):
    batch_paths = patch_paths[i:i + batch_size]
    batch_features = extractor.extract_batch_features(batch_paths)

    for path, feat in batch_features:  # feat shape: (5, D)
        # Parse x, y from filename e.g. TCGA-3L-AA1B_x2608_y14048_patch00000.png
        fname = os.path.basename(path)
        try:
            x = int(fname.split("_x")[1].split("_")[0])
            y = int(fname.split("_y")[1].split("_")[0])
        except Exception:
            print(f"Skipping coordinate extraction for {fname}")
            continue

        all_coords.append([x, y])
        all_features.append(feat.numpy())  # shape: (5, D)

# Convert to numpy arrays
all_coords_np = np.array(all_coords)         # shape: (N, 2)
all_features_np = np.array(all_features)     # shape: (N, 5, D)

print("Final feature shape:", all_features_np.shape)
print("Final coords shape:", all_coords_np.shape)

# Write to HDF5 file
with h5py.File(save_h5_path, "w") as h5f:
    h5f.create_dataset("coords", data=all_coords_np)       # shape: (N, 2)
    h5f.create_dataset("features", data=all_features_np)   # shape: (N, 5, D)

print(f"✅ HDF5 file saved to: {save_h5_path}")

Extracting features: 100%|██████████| 12/12 [25:05<00:00, 125.42s/it]

Final feature shape: (710, 5, 1536)
Final coords shape: (710, 2)
✅ HDF5 file saved to: D:\Aamir Gulzar\KSA_project2\dataset\Features_test\Uni_2h_fivecrop.h5


In [15]:
import os
import glob
from tqdm import tqdm
import torch

# Settings
patch_root = r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\test_data"
save_root = r"D:\Aamir Gulzar\KSA_project2\dataset\Features_test\Uni_2h_fivecrop3"
batch_size = 32  # Number of images per batch

extractor = UNI2ExtractorFiveCrop()

all_patch_paths = glob.glob(os.path.join(patch_root, "*", "*.png"))
num_total = len(all_patch_paths)

for i in tqdm(range(0, num_total, batch_size)):
    batch_paths = all_patch_paths[i:i + batch_size]
    features_list = extractor.extract_batch_features(batch_paths)

    for img_path, feature_tensor in features_list:
        # Prepare output path
        relative_path = os.path.relpath(img_path, patch_root)
        slide_folder, patch_filename = os.path.split(relative_path)
        patch_base = os.path.splitext(patch_filename)[0]

        save_dir = os.path.join(save_root, slide_folder)
        os.makedirs(save_dir, exist_ok=True)

        save_path = os.path.join(save_dir, patch_base + ".pt")
        torch.save(feature_tensor, save_path)  # shape: (5, D)


  0%|          | 1/210 [01:18<4:33:46, 78.60s/it]


KeyboardInterrupt: 

In [7]:
# FiveCrop feature extraction function
import glob
import os
from tqdm import tqdm
import torch

# Set paths
patch_root = r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\test_data"
save_root = r"D:\Aamir Gulzar\KSA_project2\dataset\Features_test\Uni_2h_fivecrop"

# Initialize extractor
extractor = UNI2ExtractorFiveCrop()

# Loop through all PNG patches
all_patch_paths = glob.glob(os.path.join(patch_root, "*", "*.png"))

for patch_path in tqdm(all_patch_paths):
    try:
        features = extractor.extract_fivecrop_features(patch_path)  # shape: (5, D)

        # Extract subfolder and filename
        relative_path = os.path.relpath(patch_path, patch_root)
        slide_folder, patch_filename = os.path.split(relative_path)
        patch_base = os.path.splitext(patch_filename)[0]  # remove .png

        # Create save path
        save_dir = os.path.join(save_root, slide_folder)
        os.makedirs(save_dir, exist_ok=True)

        save_path = os.path.join(save_dir, patch_base + ".pt")
        torch.save(features, save_path)
    
    except Exception as e:
        print(f"Error processing {patch_path}: {e}")

100%|██████████| 6695/6695 [07:27<00:00, 14.97it/s] 


In [11]:
import torch

def inspect_pt_file(pt_path):
    """Inspect the contents of a PyTorch .pt file"""
    print(f"Inspecting PT file: {pt_path}")
    print("=" * 50)
    
    data = torch.load(pt_path, map_location='cpu')
    
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"Key: {key}")
            if isinstance(value, torch.Tensor):
                print(f"  Type: Tensor")
                print(f"  Shape: {tuple(value.shape)}")
                print(f"  Dtype: {value.dtype}")
            else:
                print(f"  Type: {type(value)}")
                print(f"  Value: {value}")
            print()
    else:
        print("The .pt file is not a dictionary. Type:", type(data))
        print("Value preview:", str(data)[:1500])  # Print partial value


pt_path = r"D:\Aamir Gulzar\KSA_project2\dataset\Features_test\Uni_2h_fivecrop_2\TCGA-5M-AAT6_MSIH\TCGA-5M-AAT6_MSIH_x2528_y18800_patch00000.pt"
inspect_pt_file(pt_path)

Inspecting PT file: D:\Aamir Gulzar\KSA_project2\dataset\Features_test\Uni_2h_fivecrop_2\TCGA-5M-AAT6_MSIH\TCGA-5M-AAT6_MSIH_x2528_y18800_patch00000.pt
The .pt file is not a dictionary. Type: <class 'torch.Tensor'>
Value preview: tensor([[ 0.2484,  0.6537, -0.4724,  ..., -0.3257,  0.0247, -0.2856],
        [ 0.4742,  0.3480, -0.3398,  ..., -0.2549,  0.3102, -0.0355],
        [ 0.1615,  0.4483, -0.0556,  ..., -0.4179, -0.0586, -0.0378],
        [ 0.3698,  0.0509, -0.2789,  ..., -0.0538, -0.2697,  0.0940],
        [ 0.1104,  0.0061, -0.0850,  ...,  0.1443, -0.0485,  0.0090]])


In [ ]:
import torch
import os
from glob import glob

# Folder containing saved .pt feature files
pt_root = r"D:\Aamir Gulzar\KSA_project2\dataset\Features_test\Uni_2h_fivecrop"

# Find all .pt files
pt_files = glob(os.path.join(pt_root, "**", "*.pt"), recursive=True)

# Check shapes
for pt_file in pt_files:
    try:
        tensor = torch.load(pt_file)
        print(f"{pt_file}: shape = {tuple(tensor.shape)}")
        
        # Optional: check if shape is (5, D)
        if tensor.ndim != 2 or tensor.shape[0] != 5:
            print(f"❌ Unexpected shape in {pt_file}: {tuple(tensor.shape)}")

    except Exception as e:
        print(f"❌ Error loading {pt_file}: {e}")

Inspecting PT file: D:\Aamir Gulzar\KSA_project2\dataset\Features_test\Uni_2h_fivecrop\TCGA-5M-AAT6_MSIH\TCGA-5M-AAT6_MSIH_x2528_y18800_patch00000.pt
The .pt file is not a dictionary. Type: <class 'torch.Tensor'>
Value preview: tensor([[ 0.2484,  0.6536, -0.4724,  ..., -0.3257,  0.0247, -0.2856],
        [ 0.4743,  0.3480, -0.3398,  ..., -0.2548,  0.3103, -0.0356],
        [ 0.1615,  0.4483, -0.0556,  ..., -0.4179, -0.0586, -0.0378],
        [ 0.3698,  0.0509, -0.2789,  ..., -0.0538, -0.2697,  0.0940],
        [ 0.1104,  0.0061, -0.0850,  ...,  0.1443, -0.0485,  0.0090]])


In [ ]:
import os
import glob
import pandas as pd
import torch
from h5py import File as H5File
from transformers import AutoModel

def run_full_pipeline_Uni2h(
    patch_data_dir: str,
    extractor,
    slide_meta_csv: str,
    non_white_csv: str,
    output_root: str,
    batch_size: int = 64
):
    # Load slide metadata and non-white patch names
    slide_meta_df = pd.read_csv(slide_meta_csv)
    non_white_df = pd.read_csv(non_white_csv)
    valid_patch_names = set(non_white_df['patch_name'].astype(str))

    # Load TITAN model for slide embedding
    # titan = AutoModel.from_pretrained("MahmoodLab/TITAN", trust_remote_code=True)

    # Iterate through all slide folders in patch_data_dir
    slide_dirs = glob.glob(os.path.join(patch_data_dir, "*"))

    for slide_dir in slide_dirs:
        slide_name = os.path.basename(slide_dir)
        print(f"\nProcessing slide: {slide_name}")

        # Match objective power
        meta_row = slide_meta_df[slide_meta_df["renamed"].str.lower() == slide_name.lower()]
        if meta_row.empty:
            print(f"  ❌ Skipped: Slide metadata not found for {slide_name}")
            continue

        try:
            obj_power = int(meta_row.iloc[0]["Objective Power"])
            patch_size_level0 = 1024 if obj_power == 40 else 512 if obj_power == 20 else None
            if patch_size_level0 is None:
                print(f"  ❌ Unsupported objective power: {obj_power}")
                continue
        except Exception as e:
            print(f"  ❌ Error extracting objective power: {e}")
            continue

        # Filter valid patches for this slide
        all_patches = glob.glob(os.path.join(slide_dir, "*.png"))
        patch_paths = [p for p in all_patches if os.path.basename(p) in valid_patch_names]

        if not patch_paths:
            print(f"  ❌ No valid non-white patches found for {slide_name}")
            continue

        # Sort and extract features
        patch_paths = sort_by_coords(patch_paths)
        output_dir = os.path.join(output_root, slide_name)
        os.makedirs(output_dir, exist_ok=True)

        features_list = []
        for i in range(0, len(patch_paths), batch_size):
            batch = load_image_batch(patch_paths, i, batch_size, extractor.eval_transform)
            if batch.numel() == 0:
                continue
            batch = batch.to(extractor.device)
            with torch.no_grad():
                feats = extractor.extract_features(batch).cpu().numpy()
                features_list.append(feats)

        if not features_list:
            print(f"  ❌ No features extracted for {slide_name}")
            continue

        features_np = np.concatenate(features_list, axis=0)
        h5_path = create_h5_from_features(
            features_np,
            patch_paths,
            output_dir=output_dir,
            slide_name=slide_name,
            patch_size_level0=patch_size_level0
        )
        print(f"  ✅ Done: Saved H5 for {slide_name}")

        # Save a .pt version of the h5 contents
        with H5File(h5_path, 'r') as file:
            patch_features = torch.from_numpy(file['features'][:])
            patch_coords = torch.from_numpy(file['coords'][:])
            patch_size_lv0 = file['coords'].attrs['patch_size_level0']
        
        pt_data = {
            'features': patch_features,
            'coords': patch_coords,
            'patch_size_level0': patch_size_lv0
        }
        
        pt_path = h5_path.replace('.h5', '.pt')
        torch.save(pt_data, pt_path)
        
        print(f"  💾 Saved patch data to .pt at {pt_path}")

In [14]:
extractor = UNI2Extractor()

run_full_pipeline_Uni2h(
    patch_data_dir=r"D:\Aamir Gulzar\KSA_project2\dataset\patch_data",
    extractor=extractor,
    slide_meta_csv=r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\slide_metadata_filtered.csv",
    non_white_csv=r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\final_merged_without_white.csv",
    output_root=r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Uni_2h",
    batch_size=64
)


Processing slide: TCGA-3L-AA1B_nonMSIH
H5 file created successfully: D:\Aamir Gulzar\KSA_project2\dataset\Features\Uni_2h\TCGA-3L-AA1B_nonMSIH\TCGA-3L-AA1B_nonMSIH_patch_features.h5
Features shape: (1, 1878, 1536)
Coords shape: (1, 1878, 2)
  ✅ Done: Saved H5 for TCGA-3L-AA1B_nonMSIH
  💾 Saved patch data to .pt at D:\Aamir Gulzar\KSA_project2\dataset\Features\Uni_2h\TCGA-3L-AA1B_nonMSIH\TCGA-3L-AA1B_nonMSIH_patch_features.pt

Processing slide: TCGA-4N-A93T_nonMSIH
H5 file created successfully: D:\Aamir Gulzar\KSA_project2\dataset\Features\Uni_2h\TCGA-4N-A93T_nonMSIH\TCGA-4N-A93T_nonMSIH_patch_features.h5
Features shape: (1, 4240, 1536)
Coords shape: (1, 4240, 2)
  ✅ Done: Saved H5 for TCGA-4N-A93T_nonMSIH
  💾 Saved patch data to .pt at D:\Aamir Gulzar\KSA_project2\dataset\Features\Uni_2h\TCGA-4N-A93T_nonMSIH\TCGA-4N-A93T_nonMSIH_patch_features.pt

Processing slide: TCGA-5M-AAT4_nonMSIH
H5 file created successfully: D:\Aamir Gulzar\KSA_project2\dataset\Features\Uni_2h\TCGA-5M-AAT4_nonM

In [13]:
extractor = ConchFeatureExtractor()

run_full_pipeline(
    patch_data_dir=r"D:\Aamir Gulzar\KSA_project2\dataset\patch_data",
    extractor=extractor,
    slide_meta_csv=r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\slide_metadata_filtered.csv",
    non_white_csv=r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\final_merged_without_white.csv",
    output_root=r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5",
    batch_size=64
)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\datainsight\.cache\huggingface\token
Login successful

Processing slide: TCGA-3L-AA1B_nonMSIH
H5 file created successfully: D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5\TCGA-3L-AA1B_nonMSIH\patch_features\TCGA-3L-AA1B_nonMSIH_patch_features.h5
Features shape: (1, 1878, 768)
Coords shape: (1, 1878, 2)
  💾 Saved patch data to .pt at D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5\TCGA-3L-AA1B_nonMSIH\patch_features\TCGA-3L-AA1B_nonMSIH_patch_features.pt
  ✅ Done: Saved H5 and slide embedding for TCGA-3L-AA1B_nonMSIH

Processing slide: TCGA-4N-A93T_nonMSIH
H5 file created successfully: D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5\TCGA-4N-A93T_nonMSIH\patch_features\TCGA-4N-A93T_nonMSIH_patch_features.h5
Features shape: (1, 4240, 768)
Coords shape:

In [15]:
import os

def find_missing_subfolders(folder1, folder2):
    # Get subfolder names in each folder (excluding files)
    subfolders1 = set(f for f in os.listdir(folder1) if os.path.isdir(os.path.join(folder1, f)))
    subfolders2 = set(f for f in os.listdir(folder2) if os.path.isdir(os.path.join(folder2, f)))

    # Compare sets
    missing_in_folder2 = subfolders1 - subfolders2
    missing_in_folder1 = subfolders2 - subfolders1

    print(f"Subfolders in '{folder1}' but missing in '{folder2}':")
    for f in missing_in_folder2:
        print(f"  - {f}")

    print(f"\nSubfolders in '{folder2}' but missing in '{folder1}':")
    for f in missing_in_folder1:
        print(f"  - {f}")

# Example usage:
folder1 = r"D:\Aamir Gulzar\KSA_project2\dataset\patch_data"
folder2 = r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5"
find_missing_subfolders(folder1, folder2)


Subfolders in 'D:\Aamir Gulzar\KSA_project2\dataset\patch_data' but missing in 'D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5':
  - TCGA-CM-6680_nonMSIH

Subfolders in 'D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5' but missing in 'D:\Aamir Gulzar\KSA_project2\dataset\patch_data':


In [16]:
extractor = ConchFeatureExtractor()

run_full_pipeline(
    patch_data_dir=r"D:\Aamir Gulzar\KSA_project2\dataset\Test_data",
    extractor=extractor,
    slide_meta_csv=r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\slide_metadata_filtered.csv",
    non_white_csv=r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\feature_extraction\final_merged_without_white.csv",
    output_root=r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5_Missing",
    batch_size=64
)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\datainsight\.cache\huggingface\token
Login successful

Processing slide: TCGA-CM-6680_nonMSIH
  ❌ No valid non-white patches found for TCGA-CM-6680_nonMSIH


In [ ]:
import os
import h5py
import pandas as pd

def count_total_patches_in_h5(folder_path):
    total_patches = 0
    h5_file_count = 0

    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.h5'):
                h5_path = os.path.join(root, file)
                try:
                    with h5py.File(h5_path, 'r') as f:
                        features = f['features']
                        num_patches = features.shape[1]
                        total_patches += num_patches
                        h5_file_count += 1
                except Exception as e:
                    print(f"Error reading {h5_path}: {e}")
    
    print(f"\nProcessed {h5_file_count} H5 files")
    print(f"Total number of patches across all H5 files: {total_patches}")
    return total_patches

def count_rows_in_csv(csv_path):
    try:
        df = pd.read_csv(csv_path)
        num_rows = len(df)
        print(f"\nCSV File: {csv_path}")
        print(f"Number of rows in CSV: {num_rows}")
        return num_rows
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return 0

# Example usage
if __name__ == "__main__":
    folder_path = r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Conch_v1_5"  
    csv_path = r"D:\Aamir Gulzar\KSA_project2\dataset\final_merged_without_white.csv"  

    count_total_patches_in_h5(folder_path)
    count_rows_in_csv(csv_path)


Processed 417 H5 files
Total number of patches across all H5 files: 1377933

CSV File: D:\Aamir Gulzar\KSA_project2\dataset\final_merged_without_white.csv
Number of rows in CSV: 1377933


In [19]:
if __name__ == "__main__":
    folder_path = r"D:\Aamir Gulzar\KSA_project2\dataset\Features\Uni_2h"  
    csv_path = r"D:\Aamir Gulzar\KSA_project2\dataset\final_merged_without_white.csv"  

    count_total_patches_in_h5(folder_path)
    count_rows_in_csv(csv_path)


Processed 417 H5 files
Total number of patches across all H5 files: 1377933

CSV File: D:\Aamir Gulzar\KSA_project2\dataset\final_merged_without_white.csv
Number of rows in CSV: 1377933
